In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

Generate pairs

In [2]:
N = 1000 #nb of pair

#generate pairs
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)



In [371]:
def create_shallow_model():
    return nn.Sequential(
        nn.Linear(392, 784),
        nn.ReLU(),
        nn.Linear(784, 1560),
        nn.ReLU(),
        nn.Linear(1560, 98),
        nn.ReLU(),
        nn.Linear(98, 2)
    )

In [395]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(392, 784, kernel_size=4)
        self.conv2 = nn.Conv1d(784, 1568, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [393]:
def train_model(model, train_input, train_target):
    epoch = 25
    eta = 0.1
    mini_batches = 100
    
    criterion = torch.nn.CrossEntropyLoss()
    
    for e in range(0, epoch):
        sum_loss = 0
        
        for b in range(0, train_input.size(0), mini_batches):
            
            output = model(train_input.narrow(0, b, mini_batches).reshape(mini_batches, 1, -1))
            loss = criterion(output.squeeze(1), train_target.narrow(0, b, mini_batches))
            
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            
            with torch.no_grad():
                for p in model.parameters():
                    p = p - eta*p.grad
        
        
        '''for i in range(len(train_input)):
            print(train_input[i].reshape(1, -1).shape)
            output = model(train_input[i].reshape(1, -1))
            loss = criterion(output, train_target[i].unsqueeze(0))
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p = p - eta*p.grad'''
    

In [377]:
def compute_nb_errors(model, test_input, target):
    nb_errors = 0
    mini_batch_size = 100
    
    for b in range(0, test_input.size(0), mini_batch_size):
        output = model(test_input.narrow(0, b, mini_batch_size).reshape(mini_batch_size, 1, -1))
        _, predicted_class = output.max(2)
        
        for k in range(mini_batch_size):
            
            if target[b + k] != predicted_class[k]:
                nb_errors = nb_errors + 1
    
    '''for i in range(len(test_input)):
        output = model(test_input[i].reshape(1, -1))
        _, predicted_classes = output.max(1)
        if target[i] <= 0:
            nb_errors = nb_errors + 1'''

    return nb_errors

In [312]:
print(train_input[0].shape)
test = train_input[0].reshape(1, -1)
print(test.shape)
print(train_target[0].reshape(1,-1))
test2 = train_input[0].narrow(1, 2, 1)
print(test2.shape)

torch.Size([2, 14, 14])
torch.Size([1, 392])
tensor([[0]])
torch.Size([2, 1, 14])


In [373]:
def train_target_changed(train):
    t = torch.empty(train.size(0),2).zero_()
    for n in range (train.size(0)):
        t[n,int(train[n].item())] = 1
    return t

In [396]:
#model = create_shallow_model()
model = Net(10)
train_model(model, train_input, train_target)

RuntimeError: Given groups=1, weight of size 784 392 4, expected input[100, 1, 392] to have 392 channels, but got 1 channels instead

In [375]:
nb_train_errors = compute_nb_errors(model, train_input, train_target)
nb_test_errors = compute_nb_errors(model, test_input, test_target)

PREDECTED_CLASS tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [

        [0]])
PREDECTED_CLASS tensor([[0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        

        [0]])
PREDECTED_CLASS tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        

In [376]:
print('train error {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                   nb_train_errors, train_input.size(0)))

print('test error {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                   nb_test_errors, test_input.size(0)))

train error 48.90%% 489/1000
test error 47.60%% 476/1000
